# Grasping Exercises

In [ ]:
%load_ext autoreload
%autoreload 2
import os
from pydrake.geometry import StartMeshcat
from airo_drake.stations.ur3e_cart import MakeUR3eCartStation, RunAndPublishSimulation
from airo_drake.planners.planners_base import DualArmPlannerBase


In [ ]:
meshcat = StartMeshcat()

## GraspPlanner

In [ ]:
import numpy as np
from pydrake.all import PiecewisePose, RigidTransform, RotationMatrix, PiecewisePolynomial
from airo_drake.geometry import top_down_orientation


class GraspPlanner(DualArmPlannerBase):
    def __init__(self, meshcat):
        super().__init__(meshcat)
        self.planned = False

    def get_openness_trajectory(self, times):
        opened = np.array([self.gripper_max_openness])
        closed = np.array([0.0])
        right_gripper_openness = np.array([self.initial_right_openness_state[0]])
        openness_trajectory = PiecewisePolynomial.FirstOrderHold(
            [times["initial"], times["pregrasp"]], np.hstack([[right_gripper_openness], [opened]])
        )
        openness_trajectory.AppendFirstOrderSegment(times["grasp"], opened)
        openness_trajectory.AppendFirstOrderSegment(times["grasp_close"], closed)
        openness_trajectory.AppendFirstOrderSegment(times["lift"], closed)
        return openness_trajectory

    def Plan(self, context, state):
        if self.planned:
            return
        global_y_axis = np.array([0, 1, 0])
        grasp_orientation = RotationMatrix(top_down_orientation(gripper_open_direction=global_y_axis))
        grasp_position = np.array([0, 0, 0.01])
        grasp_pose = RigidTransform(grasp_orientation, grasp_position)
        pregrasp_pose = RigidTransform(grasp_orientation, grasp_position + np.array([0, 0, 0.05]))
        lift_pose = RigidTransform(grasp_orientation, grasp_position + np.array([0, 0, 0.2]))

        self.right_tcp_keyposes = {
            "initial": self.inital_right_tcp,
            "pregrasp": pregrasp_pose,
            "grasp": grasp_pose,
            "grasp_close": grasp_pose,
            "lift": lift_pose,
        }
        times = {"initial": 0.0, "pregrasp": 2.5, "grasp": 3.0, "grasp_close": 3.5, "lift": 6.0}

        def MakeLinearTrajectory(keyposes_dict, times_dict):
            keyposes = list(keyposes_dict.values())
            times = list(times_dict.values())
            pose_trajectory = PiecewisePose.MakeLinear(times, keyposes)
            return pose_trajectory

        self.right_tcp_trajectory = MakeLinearTrajectory(self.right_tcp_keyposes, times)
        self.right_openness_trajectory = self.get_openness_trajectory(times)
        self.UpdatePlanVisualization()
        self.planned = True


## Simulations

In [ ]:
def SimulateGrasp(additional_directives, planner, meshcat):
    meshcat.Delete()
    station = MakeUR3eCartStation(additional_directives)
    RunAndPublishSimulation(station, planner, meshcat)

In [ ]:
from airo_drake.downloaders.google_scanned_objects import get_google_scanned_object


def GetModelDirective(model_name):
    model_directory = get_google_scanned_object(model_name)
    model_sdf = os.path.join(model_directory, "model.sdf")

    model_directive = f"""- add_model:
            name: {model_name}
            file: file://{model_sdf}
    """
    return model_directive

### Scenario 1/5 - Cube 

In [ ]:
cube_name = "Granimals_20_Wooden_ABC_Blocks_Wagon_85VdSftGsLi"
cube_directive = GetModelDirective(cube_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(cube_directive, planner, meshcat)


### Scenario 2/5 - Box

In [ ]:
box_name = "Frozen_Scrabble_Jr"
box_directive = GetModelDirective(box_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(box_directive, planner, meshcat)

### Scenario 3/5 - Mug

In [ ]:
mug_name = "Threshold_Porcelain_Coffee_Mug_All_Over_Bead_White"
mug_directive = GetModelDirective(mug_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(mug_directive, planner, meshcat)

### Scenario 4/5 - Dinosaur

In [ ]:
dino_name = "Schleich_Allosaurus"
dino_directive = GetModelDirective(dino_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(dino_directive, planner, meshcat)

### Scenario 5/5 - Screwdriver

In [ ]:
screwdriver_name = "Craftsman_Grip_Screwdriver_Phillips_Cushion"
screw_directive = GetModelDirective(screwdriver_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(screw_directive, planner, meshcat)

## Bonus

### Plate

In [ ]:
plate_name = "Threshold_Bistro_Ceramic_Dinner_Plate_Ruby_Ring"
plate_directive = GetModelDirective(plate_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(plate_directive, planner, meshcat)

### DVD-box

In [ ]:
dvd_name = "Super_Mario_3D_World_Wii_U_Game"
dvd_directive = GetModelDirective(dvd_name)
planner = GraspPlanner(meshcat)
SimulateGrasp(dvd_directive, planner, meshcat)